In [20]:
import numpy as np
for interval in [5]:
    confusion = np.loadtxt("../results/confusion_"+str(interval)+".txt", delimiter=",", dtype=float)
    # index0 = np.arange(593,775,i)[confusion.argmax(0)]
    # index1 = np.arange(1350,1515,i)
    # output_file = open('../results/edges_'+str(i)+'.txt', 'w+')
    # print('check: ', len(index0)==len(index1))
    # for i0, i1 in zip(index0, index1):
    #     output_file.write(str(i0)+' '+str(i1)+'\n')

In [53]:
min_loop_interval = int(200 / interval -1)
T = interval
N = confusion.shape[0]
M = min_loop_interval
for i in range(N):
    confusion[i, max(i-M, 0):i+M] = 0

In [75]:
def read_kitti_pose(pose_path):
    with open(pose_path, 'r') as f:
        lines = f.readlines()
    poses = []
    for line in lines:
        pose = np.eye(4)
        pose_3x4 = np.array(line.strip('\n').split(' ')).reshape(3,4).astype(float)
        pose[:3,:] = pose_3x4
        poses.append(pose)
    return np.array(poses)

odom_poses = read_kitti_pose('/home/link/Projects/curly_slam/data/kitty05/cvo_intensity_img_gpu0_oct25_best/05.txt')
edges = [(i1*T, i0*T) for i0, i1 in enumerate(np.where(confusion>0.05, confusion, 0).argmax(1)) if i1!=0]
edges = [(i0,i1) for i0, i1 in edges if np.linalg.norm(odom_poses[i0,:-1,3]-odom_poses[i1,:-1,3])<50]
edges = sorted(edges, key=lambda x: x[0])

In [74]:
# plot ground truth and edges with plotly
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=odom_poses[:,0,3],
    y=odom_poses[:,1,3],
    z=odom_poses[:,2,3],
    mode='markers',
    marker=dict(
        size=0.5,
        color='blue',
        opacity=0.3
    )
))
for edge in edges:
    i0, i1 = edge
    # print(i0, i1, odom_poses[i0,:-1,3]-odom_poses[i1,:-1,3])
    fig.add_trace(go.Scatter3d(
        x=[odom_poses[i0,0,3], odom_poses[i1,0,3]],
        y=[odom_poses[i0,1,3], odom_poses[i1,1,3]],
        z=[odom_poses[i0,2,3], odom_poses[i1,2,3]],
        mode='lines',
        line=dict(
            color='red',
            width=2
        ),
        text=f'{i0} {i1}'
    ))
fig.show()


In [84]:
from spatialmath import *
from spatialmath.base import trnorm

gt_poses = read_kitti_pose('/home/link/Projects/curly_slam/data/kitty05/groundtruth.txt')
gt_poses = SE3([trnorm(pose) for pose in gt_poses])
lines = []
for i0, i1 in edges:
    relative_pose = gt_poses[i0].inv() * gt_poses[i1]
    line = f'{i0} {i1} '
    line += ' '.join([str(x) for x in np.array(relative_pose)[:3,:].reshape(-1)])
    lines.append(line)
edges_poses_file = '/home/link/Projects/curly_slam/data/kitty05/edges_gt_rel_poses.txt'
with open(edges_poses_file, 'w') as f:
    f.writelines('\n'.join(lines))

In [33]:
# plot matches
# import cv2
# dataset_folder = '/home/link/Projects/unified_cvo/test_data/kitti_05/image_2/'
# for i0, i1 in zip(index0, index1):
#     output_file.write(str(i0)+' '+str(i1)+'\n')
#     img1 = cv2.imread(dataset_folder+'0'*(6-len(str(i0)))+str(i0)+'.png')
#     img2 = cv2.imread(dataset_folder+'0'*(6-len(str(i1)))+str(i1)+'.png')
#     vis = np.concatenate((img1, img2), axis=0)
#     cv2.imwrite('../results/matches/'+str(i1)+'.png', vis)
